In [1]:
import cv2
import matplotlib.pyplot as plt
import numpy as np

In [28]:
# Umbralizado normal
imagen = cv2.imread("test_ocr_panels/00006_0.png", 0)
imagenUmbralizada = imagen.copy()
imagenUmbralizada[imagen>80] = 255
imagenUmbralizada[imagen<=80] = 0
contornos, _ = cv2.findContours(imagenUmbralizada, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
imagenContornos = np.zeros_like(imagen)
cv2.drawContours(imagenContornos, contornos, -1, (255, 255, 255), 1)
cv2.imshow("Imagen Umbralizada", imagenUmbralizada)
cv2.imshow("Imagen con Contornos", imagenContornos)
cv2.waitKey(0)
cv2.destroyAllWindows()
#plt.imshow(imagenContornos, cmap="gray")

In [100]:
# Umbralizado adaptativo
imagen = cv2.imread("test_ocr_panels/00006_0.png", 0)
imagenUmbralizada = cv2.adaptiveThreshold(imagen,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C,cv2.THRESH_BINARY_INV,11,3)
contornos, _ = cv2.findContours(imagenUmbralizada, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
imagenContornos = np.zeros_like(imagen)
cv2.drawContours(imagenContornos, contornos, -1, (255, 255, 255), 1)
cv2.imshow("Imagen Umbralizada", imagenUmbralizada)
cv2.imshow("Imagen con Contornos", imagenContornos)
cv2.waitKey(0)
cv2.destroyAllWindows()
#plt.imshow(imagenContornos, cmap="gray")

In [47]:
# MSER
def mser(imagenUmbralizada, imagen):
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (2,2))
    imagenUmbralizada = cv2.dilate(imagenUmbralizada, kernel)
    mser = cv2.MSER_create(delta=5, max_variation=0.2, min_area=50, max_area=900)
    polygons, _ = mser.detectRegions(imagenUmbralizada)
    copy = imagen.copy()
    rectangles = []
    for p in polygons:
        x, y, w, h = cv2.boundingRect(p)
        if (h / w > 1 and h / w <= 2):
            cv2.rectangle(copy, (x, y), (x + w, y + h), (0, 0, 255), 1)
            rectangles.append((x,y,w,h))
    cv2.imshow("Regiones detectadas por MSER", copy)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    return rectangles

imagen = cv2.imread("test_ocr_panels/00006_0.png")
rectanglesDetected = mser(imagenContornos, imagen)

In [57]:
from sklearn.linear_model import RANSACRegressor

def ransac(rectanglesDetected, imagen):
    ransac = RANSACRegressor()
    copy = imagen.copy()
    centrosX = []
    centrosY = []
    for rectangle in rectanglesDetected:
        x, y, w, h = rectangle
        center = (x + (w//2), y + (h//2))
        cv2.circle(copy, (center[0], center[1]), radius=1, color=(255, 255, 255), thickness=2)
        centrosX.append(center[0])
        centrosY.append(center[1])
    centrosX = np.array(centrosX).reshape(-1, 1)
    centrosY = np.array(centrosY)
    ransac.fit(centrosX, centrosY)
    lineX = np.array([centrosX.min()], centrosX.max())
    lineY = ransac.predict(lineX.reshape(-1, 1))
    cv2.line(copy, (int(lineX[0]), int(lineY[0])), (int(lineX[1]), int(line[1])), (0, 255, 0), 2)

    cv2.imshow("Regiones detectadas por MSER", copy)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

ransac(rectanglesDetected, imagen)